In [1]:
import pandas as pd
import numpy as np
import datetime as dt

from spacepy import coordinates as coord
from spacepy.time import Ticktock

In [2]:
base_f_loc = '/storage/silver/stfc_cg/hf832176/data/THEMIS/'

# <center> Read in Data </center>

In [3]:
!ls {base_f_loc}*.pkl

/storage/silver/stfc_cg/hf832176/data/THEMIS/tha_esa_energy_levels.pkl
/storage/silver/stfc_cg/hf832176/data/THEMIS/tha_mag.pkl
/storage/silver/stfc_cg/hf832176/data/THEMIS/tha.pkl
/storage/silver/stfc_cg/hf832176/data/THEMIS/tha_sst_energy_levels.pkl
/storage/silver/stfc_cg/hf832176/data/THEMIS/thb_mag.pkl
/storage/silver/stfc_cg/hf832176/data/THEMIS/thb.pkl
/storage/silver/stfc_cg/hf832176/data/THEMIS/thc_mag.pkl
/storage/silver/stfc_cg/hf832176/data/THEMIS/thc.pkl
/storage/silver/stfc_cg/hf832176/data/THEMIS/thd_mag.pkl
/storage/silver/stfc_cg/hf832176/data/THEMIS/thd.pkl
/storage/silver/stfc_cg/hf832176/data/THEMIS/the_mag.pkl
/storage/silver/stfc_cg/hf832176/data/THEMIS/the.pkl


In [4]:
e_levs_sst = pd.read_pickle(base_f_loc+'tha_sst_energy_levels.pkl')
display(e_levs_sst.T)
e_levs_esa = pd.read_pickle(base_f_loc+'tha_esa_energy_levels.pkl')
display(e_levs_esa.T)

e_levs = [i for i in e_levs_esa.index[::-1]]+[i for i in e_levs_sst.index]
# display(e_levs)

,31000.0,41000.0,52000.0,65500.0,93000.0,139000.0,203500.0,293000.0,408000.0,561500.0,719500.0,NaN,NaN,NaN,NaN,NaN
eV,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15


,31766.636719,26831.396484,20383.251953,15484.854492,11763.428711,8936.366211,6788.923828,5157.255371,3917.749512,2976.849854,...,83.861748,63.502251,48.474998,36.841000,28.115499,20.844250,15.996750,12.603500,9.210250,7.271250
eV,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31


In [5]:
# Read .pkl files
tha = pd.read_pickle(base_f_loc+'tha.pkl')
thb = pd.read_pickle(base_f_loc+'thb.pkl')
thc = pd.read_pickle(base_f_loc+'thc.pkl')
thd = pd.read_pickle(base_f_loc+'thd.pkl')
the = pd.read_pickle(base_f_loc+'the.pkl')

In [6]:
# reorder sst columns by ascending energy
cols = tha.columns.tolist()[:32][::-1]+['E_'+str(i+1) for i in range(16)]+tha.columns.tolist()[48:]
tha = tha[cols]
thb = thb[cols]
thc = thc[cols]
thd = thd[cols]
the = the[cols]

# Reorder esa columns by ascending energy
tha.columns = ['esa_E_'+str(i+1) for i in range(32)]+tha.columns.tolist()[32:]
thb.columns = ['esa_E_'+str(i+1) for i in range(32)]+tha.columns.tolist()[32:]
thc.columns = ['esa_E_'+str(i+1) for i in range(32)]+tha.columns.tolist()[32:]
thd.columns = ['esa_E_'+str(i+1) for i in range(32)]+tha.columns.tolist()[32:]
the.columns = ['esa_E_'+str(i+1) for i in range(32)]+tha.columns.tolist()[32:]

# <center> Transform Data to GSM Coordinates </center>

In [7]:
# Coordinate transform
def get_c_init(spacecraft):
    '''
    Given a THEMIS spacecraft dataframe, function
    extracts and formats the GSM position into list
    of [x,y,z]
       
    Returns list'''
    my_coords = [np.array([x,y,z])/6378.1
                 for x,y,z in zip(spacecraft['pos_x_gsm'],
                                  spacecraft['pos_y_gsm'],
                                  spacecraft['pos_z_gsm'])]
    return my_coords

def tttime(spacecraft):
    '''Converting from datetimes to Ticktock times
       
       Returns Ticktock time list'''
    tt = Ticktock([i for i in spacecraft.index], dtype='UTC')
    return tt

def get_c_final(spacecraft):
    '''
    Converts the GSM coordinates into GMAG coordinates.
    Returns GMAG in cartesian and spherical coordinates
    '''
    my_coords = get_c_init(spacecraft)
    cvals = coord.Coords(data = my_coords,
                         dtype = 'GSM',
                         carsph = 'car',
                         units = ['Re', 'Re', 'Re'])
    
    cvals.ticks = tttime(spacecraft)
    newcvals = cvals.convert('MAG', 'car')
    mltcvals = cvals.convert('MAG', 'sph')
    return (newcvals,mltcvals)

def c_to_df(spacecraft,scstr,base_f_loc):
    '''
    Extracts the x, y and z and r, lat, lon coordinates from
    the conversion.
    Calculates MLT from the longitute.
    Appends data to DFs and saves DFs
    '''
    newcvals, mltcvals = get_c_final(spacecraft)
    
    pos_x_mag = newcvals.x
    pos_y_mag = newcvals.y
    pos_z_mag = newcvals.z

    pos_r_mag = mltcvals.radi
    pos_lat_mag = mltcvals.lati
    pos_lon_mag = mltcvals.long

    mlt_mag = [i*12/180 if i >= 0 else (12+np.abs(i)*12/180) for i in pos_lon_mag]
    
    var_names = ['pos_x_mag','pos_y_mag','pos_z_mag',
                 'pos_r_mag','pos_lat_mag','pos_lon_mag',
                 'mlt_mag']
    variables = [pos_x_mag,pos_y_mag,pos_z_mag,
                 pos_r_mag,pos_lat_mag,pos_lon_mag,
                 mlt_mag]
    
    for i,j in zip(var_names,variables):
        spacecraft[i] = j
        
    spacecraft.to_pickle(base_f_loc+scstr+'_mag.pkl')

    return spacecraft

In [8]:
tha = c_to_df(tha,'tha',base_f_loc)
thb = c_to_df(thb,'thb',base_f_loc)
thc = c_to_df(thc,'thc',base_f_loc)
thd = c_to_df(thd,'thd',base_f_loc)
the = c_to_df(the,'the',base_f_loc)

# <center> Transform Differential Flux to PSD </center>

In [9]:
# DEF to PSD
def def_2_psd(defs,e_channel):
    m_e = 9.1093*10**-31
    c = 299792458
    eV = 1.602*10**-19
    
    defs = np.array(defs)
    defs_si = defs/(10**-6)
    
    energy = e_channel*eV
    
    psd = (defs_si*(c**2))/((energy**2)+2*m_e*(c**2)*energy)
    
    return psd

def psd_to_df(df,e_levels):
    for i in range(len(e_levels)):
        column = df.columns[i]
        at_energy = e_levels[i]
        df[column+'_psd'] = def_2_psd(df[column],at_energy)
        
    return df

In [10]:
tha = psd_to_df(tha,e_levs)
thb = psd_to_df(thb,e_levs)
thc = psd_to_df(thc,e_levs)
thd = psd_to_df(thd,e_levs)
the = psd_to_df(the,e_levs)

$\theta$ increases clockwise for y>0 between 0 and 180

$\theta$ decreases anti-clockwise for y<0 between 0 and -180

# <center> Select Equatorial Data </center>

For analysis, data is limited to the equatorial plane and radial limits about an estimate of the ORB boundary.

Equatorial region: Z = 0 $\pm$ 0.5 R$_E$.

Radial limits: 5 R$_E$ $\leqslant$ R $\leqslant$ 15 R$_E$

Dawn (Dusk): 6 (18) $\pm$ 3 MLT 

In [11]:
def how_much(df):
    equa_r = df[(np.abs(df['pos_z_mag'])<0.5) &
                 (df['pos_r_mag'] >= 5) & (df['pos_r_mag'] <= 15)]
    dawn = equa_r[(3 <= equa_r['mlt_mag']) &
                  (equa_r['mlt_mag'] <= 9)]
    dusk = equa_r[(15 <= equa_r['mlt_mag']) &
                  (equa_r['mlt_mag'] <= 21)]
    print('DF: '+str(len(df)),'EQR: '+str(len(equa_r)),'Dawn: '+str(len(dawn)),'Dusk: '+str(len(dusk)))
    return dawn,dusk

In [12]:
a_dawn, a_dusk = how_much(tha)
b_dawn, b_dusk = how_much(thb)
c_dawn, c_dusk = how_much(thc)
d_dawn, d_dusk = how_much(thd)
e_dawn, e_dusk = how_much(the)

DF: 445922 EQR: 97071 Dawn: 24391 Dusk: 22219
DF: 445520 EQR: 6904 Dawn: 1770 Dusk: 2697
DF: 445921 EQR: 22647 Dawn: 7929 Dusk: 10586
DF: 445920 EQR: 74799 Dawn: 29210 Dusk: 44490
DF: 445922 EQR: 58729 Dawn: 8492 Dusk: 40596


In [13]:
dawn = pd.concat([a_dawn,b_dawn,c_dawn,d_dawn,e_dawn],axis=0,
                 sort=False,ignore_index=False)
dusk = pd.concat([a_dusk,b_dusk,c_dusk,d_dusk,e_dusk],axis=0,
                 sort=False,ignore_index=False)
print(dawn.shape,dusk.shape)

(71792, 119) (120588, 119)


Note:

Checking the date ranges for Dawn and Dusk shows that they haven't been swapped around during processing.

In [14]:
# Reordering PSD
subset = ['esa_E_1_psd', 'esa_E_2_psd', 'esa_E_3_psd', 'esa_E_4_psd',
          'esa_E_5_psd', 'esa_E_6_psd', 'esa_E_7_psd', 'esa_E_8_psd',
          'esa_E_9_psd', 'esa_E_10_psd', 'esa_E_11_psd', 'esa_E_12_psd',
          'esa_E_13_psd', 'esa_E_14_psd', 'esa_E_15_psd', 'esa_E_16_psd',
          'esa_E_17_psd', 'esa_E_18_psd', 'esa_E_19_psd', 'esa_E_20_psd',
          'esa_E_21_psd', 'esa_E_22_psd', 'esa_E_23_psd', 'esa_E_24_psd',
          'esa_E_25_psd', 'esa_E_26_psd', 'esa_E_27_psd', 'esa_E_28_psd',
          'esa_E_29_psd', 'esa_E_30_psd', 'esa_E_31_psd',
          'E_1_psd', 'E_2_psd', 'E_3_psd', 'E_4_psd', 'E_5_psd', 'E_6_psd',
          'E_7_psd', 'E_8_psd', 'E_9_psd', 'E_10_psd', 'E_11_psd']
dawn_f = dawn.dropna(subset=subset)
dusk_f = dusk.dropna(subset=subset)

dawn_f2 = dawn_f.drop(columns=['E_12_psd','E_13_psd','E_14_psd','E_15_psd',
                               'E_16_psd','esa_E_32_psd']).copy()
dusk_f2 = dusk_f.drop(columns=['E_12_psd','E_13_psd','E_14_psd','E_15_psd',
                               'E_16_psd','esa_E_32_psd']).copy()

e_levs2 = e_levs.copy()
e_levs2 = e_levs2[:31] + e_levs2[32:-5]

In [18]:
dawn_f2.to_pickle(base_f_loc+'dawn_f2')
dusk_f2.to_pickle(base_f_loc+'dusk_f2')